In [1]:
#dev/python3
import numpy as np
import itertools

In [2]:
file = open('T10I4D100K.dat', 'r')

data = []
for line in file:
    x = list(map(int, line.split(" ")[:-1]))
    data.append(set(x))

numItems = len(data)
data

[{25,
  52,
  164,
  240,
  274,
  328,
  368,
  448,
  538,
  561,
  630,
  687,
  730,
  775,
  825,
  834},
 {39, 120, 124, 205, 401, 581, 704, 814, 825, 834},
 {35, 249, 674, 712, 733, 759, 854, 950},
 {39, 422, 449, 704, 825, 857, 895, 937, 954, 964},
 {15, 229, 262, 283, 294, 352, 381, 708, 738, 766, 853, 883, 966, 978},
 {26, 104, 143, 320, 569, 620, 798},
 {7, 185, 214, 350, 529, 658, 682, 782, 809, 849, 883, 947, 970, 979},
 {227, 390},
 {71,
  192,
  208,
  272,
  279,
  280,
  300,
  333,
  496,
  529,
  530,
  597,
  618,
  674,
  675,
  720,
  855,
  914,
  932},
 {183,
  193,
  217,
  256,
  276,
  277,
  374,
  474,
  483,
  496,
  512,
  529,
  626,
  653,
  706,
  878,
  939},
 {161, 175, 177, 424, 490, 571, 597, 623, 766, 795, 853, 910, 960},
 {125, 130, 327, 698, 699, 839},
 {392, 461, 569, 801, 862},
 {27, 78, 104, 177, 733, 775, 781, 845, 900, 921, 938},
 {101, 147, 229, 350, 411, 461, 572, 579, 657, 675, 778, 803, 842, 903},
 {71, 208, 217, 266, 279, 290, 458, 478

In [3]:
#count number of occurences per item (C1 + count)
def generateC1(dataSet):
    productDict = {}
    for basket in dataSet:
        for product in basket:
            if product not in productDict:
                productDict[product] = 1
            else:
                productDict[product] = productDict[product] + 1
    return productDict

In [4]:
#eliminate items that have a support under the threshold 
def filterBySupport(d, support):
    return {k:v for k,v in d.items() if d[k]/numItems > support}

In [5]:
#create list of lenght k
def generate_candidates_1(candidates, k):
    return list(itertools.combinations(set(candidates), k))

def generate_candidates_k(candidates, k):
    l = []
    list(l.extend(row) for row in candidates)
    return list(itertools.combinations(set(l), k))

In [14]:
def generateC2(lk, k, dataset):
    
    countDict = {}
    
    for basket in dataset:
        for candidate in lk:
            if candidate in basket:
                newcandset = basket.copy()
                newcandset.remove(candidate)
                if len(newcandset) != 0:
                    newcandidates = {(item, candidate) for item in newcandset}

                for c in newcandidates:
                    
                    if candidate not in countDict:
                        countDict[c] = 1
                    else:
                        countDict[c] +=1
                                          
    return countDict
                

In [15]:
#test

C1 = generateC1(data)
L1 = filterBySupport(C1, 0.01)
#print(L1)
print(set(L1.keys()))
C2 = generateC2(set(L1.keys()), 2, data)
print(C2)

{1, 4, 5, 6, 8, 10, 12, 17, 21, 25, 27, 28, 31, 32, 33, 35, 37, 38, 39, 41, 43, 45, 48, 51, 52, 54, 55, 57, 58, 68, 69, 70, 71, 72, 73, 75, 78, 85, 90, 93, 94, 97, 100, 104, 105, 110, 111, 112, 115, 116, 120, 122, 125, 126, 129, 130, 132, 140, 143, 145, 147, 151, 154, 157, 161, 162, 163, 168, 170, 171, 173, 175, 177, 181, 183, 185, 192, 196, 197, 198, 201, 204, 205, 207, 208, 210, 214, 217, 227, 229, 234, 236, 239, 240, 242, 258, 259, 265, 266, 274, 275, 276, 279, 280, 283, 285, 290, 294, 296, 308, 309, 310, 319, 322, 325, 326, 332, 334, 335, 336, 343, 346, 348, 349, 350, 351, 354, 357, 361, 362, 366, 368, 373, 377, 378, 381, 385, 387, 390, 392, 394, 401, 403, 405, 411, 413, 414, 419, 422, 423, 424, 427, 428, 429, 438, 440, 448, 449, 450, 458, 460, 461, 468, 469, 470, 471, 472, 477, 480, 486, 487, 489, 490, 494, 496, 500, 504, 509, 510, 511, 513, 515, 516, 517, 521, 522, 523, 526, 527, 529, 530, 534, 538, 540, 541, 546, 548, 550, 554, 561, 563, 567, 569, 571, 572, 573, 574, 576, 577, 5

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
#eliminate records where no candidates appear
def filterDataset(dataset, candidates):
    return [x for x in dataset if any(c in x for c in candidates)]

def filterDataset_k(dataset, bindexes):
    l = []
    list(l.extend(row) for row in bindexes)
    return [dataset[i] for i in l]

In [11]:
def generateLK(dataset, support):
    
    L = []
    baskets_indexes = {}
    dataset = dataset.copy()
    C = generateC1(dataset)
    L.append(filterBySupport(C, 0.05))
   
    #dataset = filterDataset(dataset, list(L[0].keys()))
    #C = generate_candidates_1(list(L[0].keys()), 2)
    
    print("Baskets: ", len(dataset))
    
    for k in range(2, 100):
        
        C = generateCk(L[k-1].keys(), k, dataset)
        L.append(filterBySupport(C, support))
        
        baskets_indexes = {bindex: baskets_indexes[bindex] for bindex in list(L[k-1].keys())}
        #dict_you_want = { your_key: old_dict[your_key] for your_key in your_keys }

        dataset = filterDataset_k(dataset, baskets_indexes)
        print("Baskets: ",len(dataset))
       
        if len(L[len(L)-1]) == 0: 
            break
        C = generate_candidates_k(list(L[k-1].keys()), k+1)
        
    return L
        

In [13]:
L = generateLK(data, 0.001)
L

Baskets:  46750
Baskets:  90
Baskets:  0


[{368: 7828,
  766: 6265,
  529: 7057,
  217: 5375,
  419: 5057,
  722: 5845,
  354: 5835,
  684: 5408,
  829: 6810,
  494: 5102},
 {(529, 217): 403,
  (217, 766): 276,
  (419, 722): 366,
  (354, 684): 219,
  (529, 829): 584,
  (684, 829): 349,
  (494, 829): 267,
  (419, 368): 355,
  (419, 829): 259,
  (368, 829): 1194,
  (368, 529): 640,
  (368, 766): 504,
  (354, 529): 301,
  (684, 766): 613,
  (368, 722): 392,
  (354, 368): 319,
  (684, 529): 334,
  (494, 217): 183,
  (354, 217): 280,
  (494, 368): 860,
  (354, 419): 263,
  (354, 722): 566,
  (494, 766): 227,
  (722, 766): 328,
  (217, 829): 275,
  (419, 684): 155,
  (368, 217): 303,
  (684, 368): 387,
  (722, 829): 294,
  (354, 829): 259,
  (722, 217): 498,
  (354, 766): 329,
  (419, 217): 344,
  (419, 766): 238,
  (684, 722): 443,
  (354, 494): 189,
  (494, 529): 225,
  (829, 766): 321,
  (529, 766): 317,
  (684, 494): 208,
  (529, 722): 283,
  (684, 217): 198,
  (494, 722): 226,
  (419, 494): 176,
  (419, 529): 252},
 {}]